# RA2CE feature: Equity analysis

In [1]:
import ast
import random
import sys
import webbrowser
import geopandas as gpd
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.pyplot import get_cmap
from matplotlib.colors import to_hex
from pathlib import Path
import rasterio

Then, we have to set some basic variables. Keep in mind that you will have to specify the path to your local machine yourself.
First, we will set the path to the ra2ce folder and we will initialize the names for the network and analysis ini files.

Afterwards, let's find the examples folder on your machine.

In [2]:
from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

_network_ini_name = "network.ini" #set the name for the network.ini
_analysis_ini_name = "analyses.ini" #set the name for the analysis.ini

folder_dir = Path(r'C:\python\ra2ce\examples\data') # Set the path to your examples folder

c:\Users\meije_le\Anaconda3\envs\ra2ce_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
root_dir = folder_dir/'equity_analysis'

network_ini = root_dir / _network_ini_name
assert network_ini.is_file()

analysis_ini = root_dir / _analysis_ini_name
assert analysis_ini.is_file()

Run RA2CE. Notice the information RA2CE gives you. Education locations are referred to as D1 (destination 1) and hospital locations as D2 (destination 2).

In [4]:
handler = Ra2ceHandler(network=network_ini, analysis=analysis_ini)
handler.configure()
handler.run_analysis()

2023-11-08 03:25:35 PM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2023-11-08 03:25:35 PM - [network_wrapper_factory.py:53] - root - INFO - Start creating a network from the submitted shapefile.
2023-11-08 03:25:35 PM - [osm_network_wrapper.py:56] - root - INFO - Start downloading a network from OSM.
2023-11-08 03:25:36 PM - [osm_network_wrapper.py:176] - root - INFO - graph downloaded from OSM with 4,036 nodes and 6,591 edges
c:\Users\meije_le\Anaconda3\envs\ra2ce_env\lib\site-packages\osmnx\simplification.py:462: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = utils_graph.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\meije_le\Anaconda3\envs\ra2ce_env\lib\site-packages\osmnx\simplification.py:510: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrec

## Inspecting results
Let's do some output exploration!

In [5]:
analysis_output_path = root_dir / "output" / "multi_link_origin_closest_destination"
gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_origins.gpkg')
gdf.head() #show the origins

2023-11-08 03:27:52 PM - [collection.py:162] - fiona._env - ERROR - C:\python\ra2ce\examples\data\equity_analysis\output\multi_link_origin_closest_destination\multilink_origin_closest_dest_origins.gpkg: No such file or directory


DriverError: C:\python\ra2ce\examples\data\equity_analysis\output\multi_link_origin_closest_destination\multilink_origin_closest_dest_origins.gpkg: No such file or directory

Notice the different columns. Especially the columns 'EV1_ma_AD1' and 'EV1_ma_AD2' are of interest. They refer to 'EV1' (event 1), maximum water depth (ma) and Destination 1 (education) or 2 (hospital).

Below, we visualise which origins have access to their closest destination, given the disruption of the road network because of the flood. 

*Note: below we visualize the access to education (D1).*

In [ ]:
gdf.explore(column='EV1_ma_AD1', cmap=['green', 'red'])

We can also check the access to the hospitals:

In [ ]:
gdf.explore(column='EV1_ma_AD2', cmap=['green', 'red'])

What RA2CE also provides are optimal routes to a destination. In this case, since we use the origin **closest** destination analysis, the routes will be centered around the destination. For every origin that still has access, RA2CE computes the optimal route to that closest destination. 

If we want to visualize this in a way that shows the routes clearly, it requires some filtering. You for example need to filter the destination of interest. You can do this here or in a GIS software.

In [ ]:
origin_gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_origins.gpkg') # read in the origins
destinations_gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_destinations.gpkg') # read in the destination
optimal_routes_gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_optimal_routes_with_hazard.gpkg') # read in the optimal routes given the hazard

In [ ]:
b_5_gdf = destinations_gdf[destinations_gdf['d_id']=='B_5'] # filter on destination B5
optimal_routes_b_5_gdf = optimal_routes_gdf[optimal_routes_gdf['destination'] == 'B_5'] # filter on destination B5
origins_with_optimal_route_b_5 = origin_gdf[origin_gdf['o_id'].isin(optimal_routes_b_5_gdf['origin'])] # both the origins and the destinations file hold destination information

optimal_routes_b_5_map = optimal_routes_b_5_gdf.explore(color='black')
b_5_map = b_5_gdf.explore(m=optimal_routes_b_5_map, color='blue', marker_kwds={'radius':10})
origins_with_optimal_route_b_5.explore(m=b_5_map, color='green', marker_kwds={'radius':5})

Notice how some origins have access but are not directly on a road. This is becasuse RA2CE makes an assumption where the origins will 'enter' the road, by projecting the origin on the nearest road vertice. 

It is also interesting to see how many people are cut-off from their nearest education location or hospital. We first show the the number of people that does not have access to any education location.

In [ ]:
origins_gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_origins.gpkg')
no_access_gdf = origins_gdf[origins_gdf['EV1_ma_AD1']=='no access']

no_access_gdf.explore(column='POPULATION', cmap='cool', marker_kwds={'radius':5}, tiles="CartoDB positron")

And now the same for the hospitals:

In [ ]:
origins_gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_origins.gpkg')
no_access_gdf = origins_gdf[origins_gdf['EV1_ma_AD2']=='no access']

no_access_gdf.explore(column='POPULATION', cmap='cool', marker_kwds={'radius':5}, tiles="CartoDB positron")

These two maps almost look the same, are they? Are there just as many people isolated from any education location as from any hospital location? Let's check it in the results.

In [ ]:
origins_gdf = gpd.read_file(analysis_output_path / 'multilink_origin_closest_dest_origins.gpkg')
d1_access_d2_no_access = origins_gdf[(origins_gdf['EV1_ma_AD1']=='access') & (origins_gdf['EV1_ma_AD2']=='no access')]
d1_no_access_d2_access = origins_gdf[(origins_gdf['EV1_ma_AD1']=='no access') & (origins_gdf['EV1_ma_AD2']=='access')]

print(f"How many population points have access to any education location and no access to any hospital? {d1_access_d2_no_access.shape[0]}")
print(f"How many population points have no access to any education location and no access to any hospital? {d1_no_access_d2_access.shape[0]}")

Indeed, as we already saw in the maps all locations have either access to an education location ánd to a hospital or no access to either.